# Download NRR geotiffs from shapefile for the desired time period

In this Notebook, data from the Nationale Regenradar (NRR) is downloaded via the Lizard platform of Nelen & Schuurmans and Royal HaskoningDHV (RHDHV) (2023). The data is downloaded via de API, with an username and password. The input is the area of interest, defined by a shapefile. The downloaded data consists of a geotiff of each time step, which will be processed to a NetCDF file in a different notebook.

<I>Note: the download takes multiple days, as the data is per 5 minutes and the period of interest is more than a year.</I>

Nelen & Schuurmans. (2023). <I>Welcome to the Lizard documentation! -- Lizard 2022.02 documentation.</I> Retrieved from: https://docs.lizard.net/a_lizard.html

## Import the required packages

In [2]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import requests
from shapely.geometry import *
import time

## Lizard password

Define your username and password

In [3]:
headers = {
    'username': '__key__',
    'password': 
}

## Define the needed formulas

In [4]:
def shape_to_bbox(shape):
    '''
    Returns the geometry, width, and height of the area of interest which is given by a shapefile
    Input: Shapefile in EPSG: 28992
    Output: Polygon geometry, and area width and height
    '''
    
    # Load the shape file of the area of interest
    gdf = gpd.read_file(shape)
    
    # NRR rasterdata is in EPSG: 28992
    if gdf.crs.to_epsg() != 28992:
        gdf.to_crs(28992, inplace=True)
        
    # Get the geometry of the input shape file
    poly = gdf.geometry[0]
    gdf['geometry'][0] = box(*tuple(map(lambda x: round(x, -3), poly.envelope.buffer(500).bounds)))
    
    # Get the width and height of the area of interest
    bounds = gdf.geometry[0].bounds
    width = (bounds[2]-bounds[0])/1000
    height = (bounds[3]-bounds[1])/1000
    
    #Convert the crs to the needed EPSG:4326
    gdf.to_crs(4326, inplace=True)
    return gdf.geometry[0],width,height

def download_result(response, outfile):
    '''
    Download the NRR rasterdata as geotiff
    Input: The link from where the data is downloaded and the location of the outputfile
    Output: NRR rasterdata geotiff
    '''
    # Download the NRR rasterdata
    f = open(outfile, 'wb')
    for chunk in response.iter_content(chunk_size=512 * 1024): 
        if chunk: # filter out keep-alive new chunks
            f.write(chunk)
    f.close()

def download_NRR_rasters(shapefile, start, end, outfolder, timestamps=None):
    '''
    Download the NRR rasterdata for the area and time period of interest
    Input: Area shapefile, start and end time of the time period, the outputfolder and if needed the failed timestamps
    Output: NRR rasterdata geotiff and the failed timestamps
    '''
    
    #Download the NRR rasters
    # Define the Lizard API url
    url = "https://rhdhv.lizard.net/api/v4/rasters/730d6675-35dd-4a35-aa9b-bfb8155f9ca7/data/?format=geotiff&geom={}&projection=EPSG:4326&width={}&height={}&start={}"
    
    # Get the shapefile geometry, width and height of the area of interst
    geom, width, height = shape_to_bbox(shapefile)
    
    # Define the timestamps of the period of interest
    if timestamps == None:
        timestamps = pd.date_range(start, end, freq='5min')
        timestamps = [t.strftime('%Y-%m-%dT%H:%M:%SZ') for t in timestamps]
    
    failed_timestamps = []    
    
    # Download the NRR rasterdata per timestep
    for t in timestamps:
        print(t)
        filename =  'NRR_' + t.replace(':', '_').replace(' ', '') + '.tif'
        curl = url.format(str(geom), width, height, t).replace(" ", "%20")
        r = requests.get(curl, headers=headers)
        if r.status_code == 200:
            download_result(r, os.path.join(outfolder, filename))
        else:
            time.sleep(3)
            r = requests.get(curl, headers=headers)
            if r.status_code == 200:
                download_result(r, os.path.join(outfolder, filename))
            else:
                time.sleep(3)
                r = requests.get(curl, headers=headers)
                if r.status_code == 200:
                    download_result(r, os.path.join(outfolder, filename))
                else:
                    failed_timestamps.append(t)
    return failed_timestamps


## Test the shape file of the area of interest

Test your shapefile, if the coordinates of the polygon and the width and height are correct

In [5]:
file = r"C:\Users\924259\Documents\NRR_input\radar_area.shp"
poly = shape_to_bbox(file)
poly

(<POLYGON ((6.15 50.616, 6.156 50.948, 5.7 50.95, 5.698 50.618, 6.15 50.616))>,
 32.0,
 37.0)

## Define the input and output folders and the time frame

Define your input folder of your shapfile, the time period, and your outputfolder for the downloaded geotiffs

In [6]:
file = r"C:\Users\924259\Documents\NRR_input\radar_area.shp"
start = '2020-07-01 00:00:00'
end = '2022-04-01 00:00:00'
outfolder = r"C:\Users\924259\Documents\NRR_output_validation"

## Download the geotiffs

Download the geotiffs with the function download_NRR_rasters()

In [7]:
%%time

failed_timestamps = download_NRR_rasters(
    shapefile=file, 
    start=start, 
    end=end, 
    outfolder=outfolder
)

failed_timestamps

184033
2020-07-01T00:00:00Z
2020-07-01T00:05:00Z
2020-07-01T00:10:00Z


KeyboardInterrupt: 

When downloads are failed, the function download_NRR_rasters() is run for the failed timestamps to complete the download

In [18]:
%%time

failed_timestamps = download_NRR_rasters(
    shapefile=file, 
    start=start, 
    end=end, 
    outfolder=outfolder,
    timestamps=failed_timestamps
)

failed_timestamps

CPU times: total: 62.5 ms
Wall time: 80.4 ms


[]